In [1]:
import pandas as pd
import numpy as np
from random import random
SEED=13

In [2]:
b_real_path="/home/samer/projects/fuzzy_sql/data/longitudinal/ready/real/b_sample.csv"
l_real_path="/home/samer/projects/fuzzy_sql/data/longitudinal/ready/real/l_sample.csv"
b_syn_path="/home/samer/projects/fuzzy_sql/data/longitudinal/ready/synthetic/b_sample_syn.csv"
l_syn_path="/home/samer/projects/fuzzy_sql/data/longitudinal/ready/synthetic/l_sample_syn.csv"

In [3]:
#Import data
b_real=pd.read_csv(b_real_path) #base
l_real=pd.read_csv(l_real_path) #longitudinal 

In [10]:
b_real

,PNUM_R,AGE,DIED,DNR,FEMALE,HOSPBRTH,Homeless,HISPANIC,NEOMAT,PL_CBSA,MEDINCST,PL_RUCA4,PL_UIC20,PL_UR_CA,PL_RUCC2,RACE,PL_NCHS2
0,21.0,82.0,0.0,0.0,1.0,0,0,2.0,0,2.0,3.0,1.0,1.0,1.0,1.0,4.0,1.0
1,105.0,83.0,0.0,0.0,0.0,0,0,2.0,0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0
2,177.0,60.0,0.0,0.0,0.0,0,0,1.0,0,2.0,3.0,1.0,1.0,1.0,1.0,3.0,1.0
3,183.0,50.0,0.0,0.0,1.0,0,0,2.0,0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0
4,291.0,77.0,0.0,0.0,0.0,0,0,2.0,0,2.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339278,14559167.0,80.0,0.0,0.0,1.0,0,0,1.0,0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0
339279,14559169.0,46.0,0.0,0.0,1.0,0,0,2.0,0,2.0,2.0,1.0,1.0,1.0,1.0,4.0,1.0
339280,14559173.0,72.0,0.0,0.0,0.0,0,0,NaN,0,2.0,4.0,1.0,2.0,2.0,2.0,NaN,3.0
339281,14559180.0,65.0,0.0,0.0,0.0,0,0,2.0,0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0


In [7]:
l_real

,PNUM_R,Date,LOS,MDC,NDX,NECODE,NPR,TOTCHG,PROCTYPE,DX1,ASCHED,AWEEKEND,DRGVER,HCUP_ED
0,21.0,5.0,3,4,8,0,0,19263.0,0,5070,0.0,1,24,4
1,21.0,9.0,1,4,3,0,2,9610.0,1,5070,1.0,0,24,0
2,105.0,6.0,3,4,10,0,0,21575.0,0,486,0.0,0,24,4
3,105.0,6.0,2,10,8,0,1,20913.0,1,27651,0.0,0,24,4
4,105.0,6.5,7,5,11,2,2,107029.0,1,42731,0.0,0,24,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890746,14559173.0,9.0,2,8,4,0,1,54798.0,1,71516,1.0,0,24,0
890747,14559180.0,7.0,4,5,4,0,3,55636.0,1,42731,0.0,1,24,4
890748,14559180.0,9.0,0,5,5,0,1,NaN,1,42731,1.0,0,24,0
890749,14559189.0,7.0,4,11,5,0,2,53653.0,1,40391,0.0,0,24,4


In [8]:
def gen_toy_syn(df, r):
    #Generate synthetic data by shuffling the columns of certain portion of the input datasets
    # r is ratio of the records to be corrupted e..g 0.2 means 20% of the table is modified.
    shuffled=df.sample(frac=1, ignore_index=True, random_state=SEED)
    r_rows=int(r*len(shuffled))
    corrupt=shuffled[0:r_rows].reset_index(drop=True)
    intact=shuffled[r_rows:].reset_index(drop=True)
    org_arr=corrupt.values
    n_col=org_arr.shape[1]
    new_arr=np.empty_like(org_arr)
    for i in range(n_col):
        new_arr[:,i]=np.random.RandomState(seed=SEED).permutation(org_arr[:,i])
    corrupt=pd.DataFrame(new_arr, columns=corrupt.columns)
    syn=pd.concat([intact, corrupt],axis=0)
    return syn


In [9]:
b_syn=gen_toy_syn(b_real, 0.2)
b_syn.to_csv(b_syn_path, index=False)
l_syn=gen_toy_syn(l_real, 0.2)
l_syn.to_csv(l_syn_path, index=False)